In [203]:
import pandas as pd
import operator

In [204]:
alp = ['а','б','в','г','д','е','ж','з','и','й','к','л','м','н','о','п','р','с','т','у','ф','х','ц','ч','ш','щ','ы','ь','э','ю','я',' ']

name_f = 'V7 (1).txt'
f = open(name_f, 'r', encoding='utf-8')
text = f.read()
f.close()

def bigrams_without_sp(text):
    count = {}
    t = 1
    while t < len(text):
        p = text[t - 1]
        l = text[t]
        if (p + l) not in count:
            count[p + l] = 1
        if (p + l) in count:  
            count[p + l] = count[p + l] + 1
        t = t + 2
    return count

a = sorted(bigrams_without_sp(text).items(), key=lambda x:x[1], reverse = True)
fr_bigrams = a[0:5]
print(fr_bigrams)

[('вм', 52), ('мь', 45), ('кч', 40), ('жг', 34), ('йв', 32)]


In [205]:
def gcd(a, b):
    if a == 0:
        return b, 0, 1
    if b == 0:
        return a, 1, 0
    else:
        r_0, r_1 = a, b 
        u_0, u_1 = 1, 0
        v_0, v_1 = 0, 1

        while r_1 != 0:
            q = r_0 // r_1
            r_0, r_1 = r_1, r_0 - q*r_1
            u_0, u_1 = u_1, u_0 - q*u_1
            v_0, v_1 = v_1, v_0 - q*v_1

    return r_0, u_0, v_0

def inv(a, b):
    return gcd(a, b)[1]

def mod_comp(a, b, n):
    d = gcd(a, n)[0]
    s = []

    if d == 1:
        return (inv(a, n) * b) % n
    elif b % d != 0: 
        return "No solutions."
    else:
        a = a / d
        b = b / d
        n = n / d
        x_0 = (inv(a, n) * b) % n

        for i in range(0, d):
            s.append(x_0 + i * n)

        return s
